In [ ]:
%pip install git+https://github.com/coax-dev/coax.git@main --quiet

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./data/tensorboard

In [1]:
# Run this cell to fix rendering errors.
import os
os.environ['SDL_VIDEODRIVER'] = 'dummy'

In [2]:
!pip install shutup
##At the top of the code
import shutup;
shutup.please()

In [ ]:
pip install coax --upgrade

In [3]:
import gymnasium
import jax
import coax
import haiku as hk
import jax.numpy as jnp
from numpy import prod
import optax


# the name of this script
name = 'sac'

# the Pendulum MDP
env = gymnasium.make('Pendulum-v1', render_mode='rgb_array')
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}")


def func_pi(S, is_training):
    seq = hk.Sequential((
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(prod(env.action_space.shape) * 2, w_init=jnp.zeros),
        hk.Reshape((*env.action_space.shape, 2)),
    ))
    x = seq(S)
    mu, logvar = x[..., 0], x[..., 1]
    return {'mu': mu, 'logvar': logvar}


def func_q(S, A, is_training):
    seq = hk.Sequential((
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(1, w_init=jnp.zeros), jnp.ravel
    ))
    X = jnp.concatenate((S, A), axis=-1)
    return seq(X)


# main function approximators
pi = coax.Policy(func_pi, env)
q1 = coax.Q(func_q, env, action_preprocessor=pi.proba_dist.preprocess_variate)
q2 = coax.Q(func_q, env, action_preprocessor=pi.proba_dist.preprocess_variate)

# target network
q1_targ = q1.copy()
q2_targ = q2.copy()

# experience tracer
tracer = coax.reward_tracing.NStep(n=5, gamma=0.9, record_extra_info=True)
buffer = coax.experience_replay.SimpleReplayBuffer(capacity=25000)
alpha = 0.2
policy_regularizer = coax.regularizers.NStepEntropyRegularizer(pi,
                                                               beta=alpha / tracer.n,
                                                               gamma=tracer.gamma,
                                                               n=[tracer.n])

# updaters (use current pi to update the q-functions and use sampled action in contrast to TD3)
qlearning1 = coax.td_learning.SoftClippedDoubleQLearning(
    q1, pi_targ_list=[pi], q_targ_list=[q1_targ, q2_targ],
    loss_function=coax.value_losses.mse, optimizer=optax.adam(1e-3),
    policy_regularizer=policy_regularizer)
qlearning2 = coax.td_learning.SoftClippedDoubleQLearning(
    q2, pi_targ_list=[pi], q_targ_list=[q1_targ, q2_targ],
    loss_function=coax.value_losses.mse, optimizer=optax.adam(1e-3),
    policy_regularizer=policy_regularizer)
soft_pg = coax.policy_objectives.SoftPG(pi, [q1_targ, q2_targ], optimizer=optax.adam(
    1e-3), regularizer=coax.regularizers.NStepEntropyRegularizer(pi,
                                                                 beta=alpha / tracer.n,
                                                                 gamma=tracer.gamma,
                                                                 n=jnp.arange(tracer.n)))


# train
while env.T < 100000:
    s, info = env.reset()

    for t in range(200):#(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, truncated, info = env.step(a)

        # trace rewards and add transition to replay buffer
        tracer.add(s, a, r, done)
        while tracer:
            buffer.add(tracer.pop())

        # learn
        if len(buffer) >= 5000:
            transition_batch = buffer.sample(batch_size=128)

            # init metrics dict
            metrics = {}

            # flip a coin to decide which of the q-functions to update
            qlearning = qlearning1 if jax.random.bernoulli(q1.rng) else qlearning2
            metrics.update(qlearning.update(transition_batch))

            # delayed policy updates
            if env.T >= 7500 and env.T % 4 == 0:
                metrics.update(soft_pg.update(transition_batch))

            env.record_metrics(metrics)

            # sync target networks
            q1_targ.soft_update(q1, tau=0.001)
            q2_targ.soft_update(q2, tau=0.001)

        if done or truncated:
            break

        s = s_next

    # generate an animated GIF to see what's going on
    if env.period(name='generate_gif', T_period=10000) and env.T > 5000:
        T = env.T - env.T % 10000  # round to 10000s
        coax.utils.generate_gif(
            env=env, policy=pi, filepath=f"./data/gifs/{name}/T{T:08d}.gif")


INFO:TrainMonitor:ep: 1,	T: 201,	G: -1.47e+03,	avg_r: -7.35,	avg_G: -1.47e+03,	t: 200,	dt: 5.176ms
INFO:TrainMonitor:ep: 2,	T: 402,	G: -1.27e+03,	avg_r: -6.35,	avg_G: -1.37e+03,	t: 200,	dt: 2.581ms
INFO:TrainMonitor:ep: 3,	T: 603,	G: -1.35e+03,	avg_r: -6.77,	avg_G: -1.37e+03,	t: 200,	dt: 3.265ms
INFO:TrainMonitor:ep: 4,	T: 804,	G: -1.25e+03,	avg_r: -6.23,	avg_G: -1.34e+03,	t: 200,	dt: 3.171ms
INFO:TrainMonitor:ep: 5,	T: 1,005,	G: -1.75e+03,	avg_r: -8.73,	avg_G: -1.42e+03,	t: 200,	dt: 2.730ms
INFO:TrainMonitor:ep: 6,	T: 1,206,	G: -967,	avg_r: -4.83,	avg_G: -1.34e+03,	t: 200,	dt: 2.598ms
INFO:TrainMonitor:ep: 7,	T: 1,407,	G: -1.76e+03,	avg_r: -8.82,	avg_G: -1.4e+03,	t: 200,	dt: 2.560ms
INFO:TrainMonitor:ep: 8,	T: 1,608,	G: -882,	avg_r: -4.41,	avg_G: -1.34e+03,	t: 200,	dt: 2.728ms
INFO:TrainMonitor:ep: 9,	T: 1,809,	G: -1.43e+03,	avg_r: -7.16,	avg_G: -1.35e+03,	t: 200,	dt: 2.632ms
INFO:TrainMonitor:ep: 10,	T: 2,010,	G: -1.28e+03,	avg_r: -6.39,	avg_G: -1.34e+03,	t: 200,	dt: 2.775ms
INFO:Tra